In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import func_proc_filepath as mFILE
import func_proc_image as mIMAGE
import cv2

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def read_and_adjust_label(file_label):
    df_label = pd.read_csv(file_label, sep=" ", header=None)
    df_label.columns = ["name", "id", "top", "left", "right", "bottom", "conf", "label"]
    df_label["file"] = df_label["name"].str.replace(".jpg", "_") + df_label['id'].map('{:02}'.format) + ".jpg"
    df_label["center_x"] = (df_label["left"] + df_label["right"])*0.5
    df_label["center_y"] = (df_label["top"] + df_label["bottom"])*0.5
    """
    ABAW: 0:Neutral, 1:Anger,   2:Disgust, 3:Fear,  4:Happiness, 5:Sadness,  6:Surprise
    ExpW: 0:angry,   1:disgust, 2:fear,    3:happy, 4:sad,       5:surprise, 6:neutral
    """
    df_label["expr"] = 0
    df_label.loc[df_label.loc[:,"label"]==0, "expr"] = 1
    df_label.loc[df_label.loc[:,"label"]==1, "expr"] = 2
    df_label.loc[df_label.loc[:,"label"]==2, "expr"] = 3
    df_label.loc[df_label.loc[:,"label"]==3, "expr"] = 4
    df_label.loc[df_label.loc[:,"label"]==4, "expr"] = 5
    df_label.loc[df_label.loc[:,"label"]==5, "expr"] = 6
    df_label.loc[df_label.loc[:,"label"]==6, "expr"] = 0
    df_label["Neutral"]=0
    df_label.loc[df_label.loc[:,"expr"]==0, "Neutral"] = 1
    df_label["Anger"]=0
    df_label.loc[df_label.loc[:,"expr"]==1, "Anger"] = 1
    df_label["Disgust"]=0
    df_label.loc[df_label.loc[:,"expr"]==2, "Disgust"] = 1
    df_label["Fear"]=0
    df_label.loc[df_label.loc[:,"expr"]==3, "Fear"] = 1
    df_label["Happiness"]=0
    df_label.loc[df_label.loc[:,"expr"]==4, "Happiness"] = 1
    df_label["Sadness"]=0
    df_label.loc[df_label.loc[:,"expr"]==5, "Sadness"] = 1
    df_label["Surprise"]=0
    df_label.loc[df_label.loc[:,"expr"]==6, "Surprise"] = 1

    return df_label


In [ ]:
def main():
    # root folder
    dir_root = str(Path(Path().resolve()).parent)

    # input: folder path including original images
    dir_img = dir_root + "\\src\\expr_image"
    # input: filepath of label
    file_label = dir_root + "\\parameters\\expr_label.lst"
    # outpu: filepath of adjusted label
    file_label_out = dir_root + "\\parameters\\expr_label.csv"

    # output: folder path
    dir_out = dir_root + "\\dataset\\expw_images\\cropped"
    if os.path.isdir(dir_out) == False:
        os.makedirs(dir_out)
        
    # read and adjust label
    df_label = read_and_adjust_label(file_label)
    # save adjusted label
    df_label_out = df_label.loc[:, ["file","expr","Neutral","Anger","Disgust","Fear","Happiness","Sadness","Surprise"]]
    df_label_out.to_csv(file_label_out, index=False)

    in_file_list = dir_img + "\\" + df_label["name"].values
    out_file_list = dir_out + "\\" + df_label["file"].values

    IMG_SIZE=300
    TH_CONF=0.89
    
    #cnt = 0
    max_cnt = len(in_file_list)
    #max_cnt = 100
    
    for cnt in range(max_cnt):
        fp = dir_img + "\\" + df_label.at[cnt, "name"]
        # get filename without ext
        name = os.path.splitext(os.path.basename(fp))[0]
        expr = df_label.at[cnt, "expr"]
        """
        ABAW: 0:Neutral, 1:Anger,   2:Disgust, 3:Fear,  4:Happiness, 5:Sadness,  6:Surprise
        ExpW: 0:angry,   1:disgust, 2:fear,    3:happy, 4:sad,       5:surprise, 6:neutral
        """
        lbx = df_label.at[cnt, "center_x"]
        lby = df_label.at[cnt, "center_y"]
        
        log = "{0}/{1}, expr:{2}, name:{3}".format(cnt, max_cnt, expr, name)
        print(log)

        if os.path.isfile(fp):
            img = cv2.cvtColor(cv2.imread(fp), cv2.COLOR_BGR2RGB)
            res_list = mIMAGE.split_image_per_face(img, base_size=IMG_SIZE, th_conf=TH_CONF)
            #res_list = mIMAGE.split_image_per_face_pad_circle(img, base_size=IMG_SIZE, th_conf=TH_CONF, pad=0.05, back=False)

            length = len(res_list)
            if length>0:
                best_l = 0
                best_dis = 99999999
                for l in range(length):            
                    imx = res_list[l][1]
                    imy = res_list[l][2]
                    dis = (lbx-imx)**(2) + (lby-imy)**(2)
                    if dis<best_dis:
                        best_dis = dis
                        best_l = l
                best_img = res_list[best_l][0]
                cv2.imwrite(out_file_list[cnt], best_img)

                if cnt < 20:
                    mIMAGE.display_cv_image(best_img, format='.png')
        
    

In [ ]:
if __name__ == "__main__":
    main()